## Imports

In [146]:
# Read stocks
import yfinance as yf

# For reading properties
from jproperties import Properties

## Load properties

In [147]:
configs = Properties()

with open('config/yf_wacc.properties', 'rb') as config_file:
     configs.load(config_file)

TICKER = configs.get('TICKER').data
# Market return is read as a string; we need to convert it to float
MKT_RETURN = float(configs.get('MKT_RETURN').data)
BOND_TICKER = configs.get('BOND_TICKER').data

## Read Ticker data

In [148]:
ticker = yf.Ticker(TICKER)

# US Treasury Yield 10 years
tnx = yf.Ticker(BOND_TICKER)

income_stmt_df = ticker.income_stmt
balance_sheet_df = ticker.balance_sheet

## Returns the latest non NaN value

In [149]:
def get_latest_non_nan(series):
    # Get the index of the first non-NaN value
    first_non_nan_index = series.first_valid_index()

    # Get the first non-NaN value
    return series[first_non_nan_index]

## Cost of Debt Calculation

In [150]:
# Interest Expense
interest_expense = get_latest_non_nan(income_stmt_df.loc['Interest Expense'])

# Income Tax Expense
income_tax = get_latest_non_nan(income_stmt_df.loc['Tax Provision'])

# Income before tax
income_before_tax = get_latest_non_nan(income_stmt_df.loc['Pretax Income'])

# Current debt
current_debt = get_latest_non_nan(balance_sheet_df.loc['Current Debt'])

# Long term debt
long_term_debt = get_latest_non_nan(balance_sheet_df.loc['Long Term Debt'])

# Cost of debt
cost_of_debt = interest_expense / (current_debt + long_term_debt)

# Effective tax rate
effective_tax_rate = income_tax / income_before_tax

# Cost of debt after tax
cost_of_debt_after_tax = cost_of_debt * (1 - effective_tax_rate)

## Cost of Equity Calculation

In [151]:
# Yield rate
risk_free_rate = tnx.info['regularMarketDayHigh']/100

# Beta
beta = ticker.info['beta']

# Cost of equity
cost_of_equity = risk_free_rate + (beta * (MKT_RETURN - risk_free_rate))             

## Weight of Debt and Equity Calculation

In [152]:
total_debt = current_debt + long_term_debt

market_cap = ticker.info['marketCap']
total = total_debt + market_cap

# Calculate the weight of debt and equity
weight_of_debt = total_debt / total
weight_of_equity = market_cap / total

## WACC Calculation

In [158]:
wacc = (weight_of_debt * cost_of_debt_after_tax) + (weight_of_equity * cost_of_equity)
'{:.2%}'.format(round(wacc,4)) + ' for ticker: ' + TICKER

'7.99% for ticker: MSFT'